In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import os

#### Web Page Elements 
Videos_List : style-scope ytd-rich-item-renderer

title --  //*[@id="video-title-link"]

views -- //*[@id="metadata-line"]/span[1]

day_posted -- //*[@id="metadata-line"]/span[2]
video_link -- //*[@id="thumbnail"]


In [ ]:
# PATH = r'chromedriver.exe'
# service = Service(executable_path=PATH)
# wd = webdriver.Chrome(service=service)
# url = "https://www.youtube.com/@krishnaik06/videos"
# wd.get(url)
# videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        
        
# for video in videos:
#     title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").get_attribute('href')
#     print(title)


## Scrolling Till 50 Links are reached


In [91]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, 1000);")
    time.sleep(5)

def fetch_video_details(url, max_videos=50):
    PATH = r'chromedriver.exe'
    service = Service(executable_path=PATH)
    wd = webdriver.Chrome(service=service)
    wd.get(url)
    
    video_details = []
    while len(video_details) < max_videos:
        videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        scroll_to_end(wd)
        
        for video in videos:
            title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").text
            view = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[1]").text
            date = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[2]").text
            link = video.find_element(By.XPATH, ".//*[@id='video-title-link']").get_attribute('href') 
            # duration = video.find_element(By.CLASS_NAME, "style-scope ytd-thumbnail-overlay-time-status-renderer").text
        

            details = {
                'title': title,
                'views': view,
                'date': date,
                'yt_link':link
            }
            
            video_details.append(details)
            if len(video_details) >= max_videos:
                break
    
    wd.quit()
    return pd.DataFrame(video_details)

url = "https://www.youtube.com/@krishnaik06/videos"
max_videos = 50
video_details_df = fetch_video_details(url, max_videos)


## Likes, Dislikes, Description, Title, Comment 

In [94]:
urls = video_details_df['yt_link']
# print(urls[0])
PATH = r'chromedriver.exe'
service = Service(executable_path=PATH)
wd = webdriver.Chrome(service=service)

wd.get(urls[0])
time.sleep(2)

likes = wd.find_element(By.XPATH,"//*[@id='segmented-like-button']/ytd-toggle-button-renderer/yt-button-shape/button/div[2]/span").text
wd.quit()
print(likes)



WebDriverException: Message: unknown error: cannot determine loading status
from no such window
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF64D7652A2+57122]
	(No symbol) [0x00007FF64D6DEA92]
	(No symbol) [0x00007FF64D5AE25D]
	(No symbol) [0x00007FF64D59C80C]
	(No symbol) [0x00007FF64D59AA3B]
	(No symbol) [0x00007FF64D59B31E]
	(No symbol) [0x00007FF64D5A86AA]
	(No symbol) [0x00007FF64D5B7F31]
	(No symbol) [0x00007FF64D5BC0DA]
	(No symbol) [0x00007FF64D59B8E7]
	(No symbol) [0x00007FF64D5B7BC3]
	(No symbol) [0x00007FF64D61EBE9]
	(No symbol) [0x00007FF64D606DB3]
	(No symbol) [0x00007FF64D5DD2B1]
	(No symbol) [0x00007FF64D5DE494]
	GetHandleVerifier [0x00007FF64DA0EF82+2849794]
	GetHandleVerifier [0x00007FF64DA61D24+3189156]
	GetHandleVerifier [0x00007FF64DA5ACAF+3160367]
	GetHandleVerifier [0x00007FF64D7F6D06+653702]
	(No symbol) [0x00007FF64D6EA208]
	(No symbol) [0x00007FF64D6E62C4]
	(No symbol) [0x00007FF64D6E63F6]
	(No symbol) [0x00007FF64D6D67A3]
	Ba

In [86]:
block = soup.find('div',attrs={'id':'title'})
print(block)

<div class="text-shell skeleton-bg-color" id="title"></div>


In [82]:
video_title = soup.find(id='title')
print(video_title.text)

In [35]:
print(soup.find('div#above-the-fold'))

None
